### Система алертинга о критических падениях трафика вчерашнего дня

In [ ]:
import pygsheets
from pprint import pprint
import httplib2
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from pandas import DataFrame
import pyodbc
import pandas.io.sql as psql
from urllib import parse
import re
from datetime import datetime
from datetime import datetime, timedelta
import statistics
from scipy import stats
from telegram import Bot
import telegram
import asyncio
import nest_asyncio
nest_asyncio.apply()
#импортируем основные библиотеки

In [2]:
engine = create_engine(
    'postgresql+psycopg2://{user}:{password}@{host}:{port}/{account}'.format(
        user='postgres',
        password=parse.quote('пароль'),
        host='хост',
        port='порт',
        account='аккаунт',
    )
)
connection = engine.connect()
# создадим строку подключения к БД

In [3]:
gc = gspread.service_account(filename='client_secret.json') #создаем объект gs для доступа к Google Sheets API

In [9]:
daterange = pd.date_range((datetime.today() - timedelta(days=65)), (datetime.today() - timedelta(days=1)))
#создадим необходимый диапазон дат для анализа - 3 месяца (65 дней) от текущей даты

In [10]:
daterange = daterange.normalize() #нормализуем диапазон, приведя временную метку к 0:00:00

In [11]:
yesterday = (datetime.today() - timedelta(days=1)).weekday() #получаем вчерашний день недели (анализируем трафик за вчера)

In [12]:
dates = [date.strftime('%Y-%m-%d') for date in daterange if date.weekday() == yesterday]
# получим список дат - аналогичных дней недели за 3 месяца, т.к. анализируем трафик за аналогичные дни недели, чтобы избежать влияния сезонности

In [13]:
days = ", ".join (str(id) for id in dates)# переведем список дат в строковый формат, чтобы передать их в SQL-запрос

In [14]:
formatted_days = "'" + days.replace(", ", "', '") + "'"# уберем лишние символы в списке, чтобы запрос не ругался

In [15]:
visions = f'''select product_stat_by_date.date, products.company_id, companies.title as company, sum(product_stat_by_date.list_count) as Показы
                from product_stat_by_date
                left join products ON products.id = product_stat_by_date.product_id
                left join companies ON companies.id = products.company_id
                where product_stat_by_date.date IN ({formatted_days}) 
                and products.company_id IN (
                    select distinct(company_id) 
                    from order_packages_v3 where now()::date<exp_date
                    )
                group by product_stat_by_date.date, products.company_id, companies.title 
                order by product_stat_by_date.date, products.company_id             
                '''
# напишем текст SQL запроса в БД. Выгружаем просмотры по компаниям за все даты, аналогичные вчерашней (если вчера был понедельник, то выгрузка вернет все понедельники за 3 мес)

In [11]:
df_visions = pd.io.sql.read_sql(visions, con = engine)# выполняем запрос и записываем в датафрейм df_visions

In [12]:
df_visions

,date,company_id,company,Показы
0,2023-08-17,7412,ООО Регион-Снабжение,3855
1,2023-08-17,7526,Компания КрасКо,2875
2,2023-08-17,21704,Альянс-Строительные Технологии,1279
3,2023-08-17,25109,Фасад-Керамика,9753
4,2023-08-17,29839,"Компания ЭКООКНА, ООО",7503
...,...,...,...,...
2524,2023-10-19,214929,Азбука Строителя,4377
2525,2023-10-19,214943,PRIDE WOOD,522
2526,2023-10-19,215000,Оптомарт,21089
2527,2023-10-19,215026,"ООО ПФК ""АРВИК""",8165


In [13]:
top_cl_lst = df_visions['company_id'].tolist()
top_cl_lst = ",".join(str(id) for id in top_cl_lst)# сформируем список уникальных компаний из датафрейма просмотров

In [14]:
paypers = f'''select id, paypercall as звонки, payperclick as Клики, paypercart as Корзина, cpa, payper_call_v2 as ЗВОНКИ, opt as ОПТ
            from companies
            where id IN ({top_cl_lst})
            '''
# напишем текст запроса в БД для выгрузки тарифов этих компаний

In [15]:
df_paypers = pd.io.sql.read_sql(paypers, con = engine)# выполняем запрос и записываем в датафрейм df_paypers

In [16]:
df_paypers

,id,звонки,Клики,Корзина,cpa,ЗВОНКИ,ОПТ
0,7412,False,True,False,False,False,False
1,7526,False,True,False,False,False,None
2,21704,False,True,False,False,False,None
3,25109,False,True,False,False,False,False
4,29839,False,False,False,False,True,True
...,...,...,...,...,...,...,...
256,214929,False,True,False,False,False,False
257,214943,False,False,False,False,True,False
258,215000,False,True,False,False,False,False
259,215026,False,False,False,False,True,True


In [17]:
df_result = pd.DataFrame(columns=['id', 'headers'])# создадим пустой датафрейм для записи тарифа из df_paypers

In [16]:
for index, row in df_paypers.iterrows():
    id = row['id']
    headers = []  
    
    for column in df_paypers.columns[1:]:
        if row[column] == True:
            headers.append(column)  
    
    df_result = df_result.append({'id': id, 'headers': ', '.join(headers)}, ignore_index=True)
# развернем датафрейм с тарифами и получим название тарифа для каждой компании

In [19]:
df_visions = pd.merge(df_visions, df_result, left_on = 'company_id', right_on = 'id', how = 'left')
# объединим датафреймы с просмотрами и тарифами по id компании

In [20]:
df_visions

,date,company_id,company,Показы,id,headers
0,2023-08-17,7412,ООО Регион-Снабжение,3855,7412,Клики
1,2023-08-17,7526,Компания КрасКо,2875,7526,Клики
2,2023-08-17,21704,Альянс-Строительные Технологии,1279,21704,Клики
3,2023-08-17,25109,Фасад-Керамика,9753,25109,Клики
4,2023-08-17,29839,"Компания ЭКООКНА, ООО",7503,29839,"ЗВОНКИ, ОПТ"
...,...,...,...,...,...,...
2524,2023-10-19,214929,Азбука Строителя,4377,214929,Клики
2525,2023-10-19,214943,PRIDE WOOD,522,214943,ЗВОНКИ
2526,2023-10-19,215000,Оптомарт,21089,215000,Клики
2527,2023-10-19,215026,"ООО ПФК ""АРВИК""",8165,215026,"ЗВОНКИ, ОПТ"


In [21]:
df_visions['date'] = pd.to_datetime (df_visions['date'], format = '%Y-%m-%d')# приведем столбец date к формату даты

In [22]:
df_visions=df_visions[df_visions['date'].dt.weekday == yesterday] #отфильтруем датафрейм, вытащив только нужные дни недели

In [23]:
vision_pivot = df_visions.pivot_table(index = ['company_id','company'], columns = 'date', values = 'Показы', aggfunc = 'sum').reset_index()
#сформируем сводную с просмотрами, чтобы компании страли строками, а латы - столбцами

In [24]:
#df_visions['date'] = pd.to_datetime (df_visions['date'], format = '%Y-%m-%d')

In [25]:
date_cols = pd.to_datetime(vision_pivot.columns, errors='coerce')
vision_pivot.columns = date_cols.strftime('%Y-%m-%d').where(date_cols.notnull(), vision_pivot.columns)
# приведем все даты в заголовках столбцов в формат даты без времени

In [26]:
vision_pivot=vision_pivot.fillna(0)# заменим значения nan на нули

In [27]:
vision_pivot['company_id']=vision_pivot['company_id'].astype(str)# приведем столбец company_id к текстовому формату

In [28]:
vision_pivot['Total'] = vision_pivot.sum(axis=1, numeric_only=True)# добавим столбец с суммой в разрезе компаний

In [29]:
vision_pivot = vision_pivot.sort_values(by = 'Total', ascending= False)# отсортируем датафрейм по убыванию итоговой цифры просмотров

In [30]:
vision_pivot = vision_pivot[vision_pivot['Total']>0] # уберем компании с нулевыми просмотрами

In [31]:
total = vision_pivot['Total'].sum()# посчитаем итоговую сумму и запишем в переменную total

In [32]:
vision_pivot['cumulative'] = vision_pivot['Total'].cumsum()# посчитаем кумулятивную сумму просмотров для ABC - анализа
vision_pivot['cum_rate'] = round(vision_pivot['Total']/total*100, 2)# посчитаем долю просмотров для АВС

In [33]:
vision_pivot['cumulative_rate'] = vision_pivot['cum_rate'].cumsum()# посчитаем кумулятивную долю просмотров для АВС

In [34]:
vision_pivot['group'] = vision_pivot['cumulative_rate'].apply(lambda x:'A' if x < 80 else ('B' if x>= 80 and x<95 else 'C') )
# присвоим группы в рамках АВС - анализа

In [35]:
#vision_pivot.drop(["Total", "cumulative", "cum_rate", "cumulative_rate"], axis = 1, inplace = True)# удалим лишние столбцы

In [36]:
vision_pivot = vision_pivot[ ['group'] + [ col for col in vision_pivot.columns if col != 'group' ] ]# оставим только нужные столбцы

In [37]:
vision_pivot

date,group,company_id,company,2023-08-17,2023-08-24,2023-08-31,2023-09-07,2023-09-14,2023-09-21,2023-09-28,2023-10-05,2023-10-12,2023-10-19
10,A,44200,"Компания ВсеИнструменты, ООО",211205.0,228941.0,234867.0,208827.0,228485.0,199318.0,192203.0,199779.0,186418.0,200589.0
74,A,190151,ООО«МосОптТорг»,39980.0,44291.0,44941.0,41404.0,44327.0,39991.0,37159.0,37726.0,36353.0,29986.0
117,A,194393,Леруа Мерлен Москва,20952.0,19825.0,24799.0,17760.0,27762.0,28625.0,14246.0,19204.0,16797.0,17452.0
53,A,173759,Grand Line (ГрандЛайн),28042.0,28447.0,26965.0,14354.0,20083.0,18061.0,17309.0,15127.0,13996.0,13498.0
27,A,83627,Магазин MirCli,14899.0,18282.0,18006.0,19982.0,19233.0,18066.0,15122.0,16049.0,17378.0,15181.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,C,214618,ООО «СРГ-ЕКАТЕРИНБУРГ»,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20.0,6.0,2.0
197,C,211143,СК ДОМ-РЕГИОН,2.0,0.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,5.0
198,C,211149,QA QA,0.0,0.0,0.0,2.0,6.0,1.0,11.0,1.0,1.0,0.0
76,C,191368,ООО «ПЕНОПЛЭКС СПб»,0.0,0.0,7.0,0.0,0.0,0.0,1.0,2.0,3.0,2.0


### Расчет статистического z-критерия для идентификации критичного падения трафика.

#### Кратко о z-критерии:
Статистически - z-score говорит нам, на сколько стандартных отклонений значение отличается от среднего по ряду

НОРМОЙ СЧИТАЕТСЯ ПОКАЗАТЕЛЬ ДО 2 СТАНДАРТНЫХ ОТКЛОНЕНИЙ

Cтандартное отклонение - это мера того, насколько разбросан набор данных. Вычисляя его, можно узнать, являются ли числа близкими к среднему значению или далеки от него.

Если точки данных находятся далеко от среднего значения, то в наборе данных имеется большое отклонение; таким образом, чем больше разброс данных, тем выше стандартное отклонение

Если модуль z-score<=2,   условно число принадлежит ряду и не является аномалией. Если показатель больше 2, то число - явная аномалия в ряду и на нее надо обратить внимание.

В данном анализе пороговое значение z-критерия понижено до 1.9


### Методика анализа

Мы сравниваем трафик за вчерашний день с трафиком за аналогичные дни за 3 месяца назад (если вчера был понедельник, то 
сравниваем с трафиком за все понедельники за 3 месяца, чтобы избежать влияния сезонности по дням недели). Если z-score больше 1.9, то вчерашнее падение трафика является критичным по сравнению с предыдущими аналогичными днями недели


In [39]:
def check_deviation(row):    
    values = [int(float(x)) for x in row[-(len(row)-3):]] #формируем список за предыдыущие периоды, приводим к целочисленному формату
    last = values[-1] #записываем в переменную просмотры за вчерашний день
    try:
        meann = statistics.mean([value for value in values[:-1] if value != 0])# считаем среднее по историческим данным
    except:
        return 0
    mediann = statistics.median([value for value in values[:-1] if value != 0]) # считаем медиану по историческим данным (не используем пока)
    if last < meann: #если прошлое меньше среднего за предыдущие периоды
        deviation = last - meann #считаем отклонение
        std = np.std(values) #считаем стандартное отклонение
        z_score = deviation / std #считаем z-критерий 
        crit_value = 1.9 # записываем в переменную пороговое значение z-критерия
        if abs(z_score) >= crit_value: #если модуль z-критерия больше или равен порогу:
            return 'Отклонение с высокой значимостью', meann, mediann, (last/meann-1)*100, (last/mediann-1)*100, z_score
# возвращаем признак значительного отклонения, среднее, медиану, отклонение от среднего и отклонение от медианы
        else:
            return 'Незначительное отклонение', meann, mediann, (last/meann-1)*100, (last/mediann-1)*100, z_score
# возвращаем признак незначительного отклонения, среднее, медиану, отклонение от среднего и отклонение от медианы
    else:
        return 0, meann, mediann, (last/meann-1)*100, (last/mediann-1)*100, 0 #возвращаем 0
vision_pivot[['significant_deviation', 'Среднее', 'Медиана', 'Отклонение от средней', 'Отклонение от медианы', 'z_score']] = vision_pivot.apply(check_deviation, axis=1, result_type='expand')
# применяем функцию, добавляем в датафрейм столбцы

In [40]:
high_descrease = vision_pivot.query('significant_deviation =="Отклонение с высокой значимостью"').sort_values(by = ['group','Отклонение от средней'])
# фильтруем датафрейм от незначительных отклонений (оставляем только значительные для передачи в таблицу и в телеграм-бот)

In [41]:
today = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
high_descrease['update'] = today# добавляем столбец - дату обновления отчета

In [42]:
high_descrease

date,group,company_id,company,2023-08-17,2023-08-24,2023-08-31,2023-09-07,2023-09-14,2023-09-21,2023-09-28,2023-10-05,2023-10-12,2023-10-19,significant_deviation,Среднее,Медиана,Отклонение от средней,Отклонение от медианы,z_score,update
160,A,206545,Парадиз Керамика,2326.0,2844.0,2637.0,2362.0,2396.0,2077.0,2212.0,1947.0,2407.0,1453.0,Отклонение с высокой значимостью,2356.444444,2362.0,-38.339306,-38.484335,-2.484120,2023-10-20 09:28:46
74,A,190151,ООО«МосОптТорг»,39980.0,44291.0,44941.0,41404.0,44327.0,39991.0,37159.0,37726.0,36353.0,29986.0,Отклонение с высокой значимостью,40685.777778,39991.0,-26.298570,-25.018129,-2.462044,2023-10-20 09:28:46
29,A,92439,"Интернет-магазин ""Ziwo""",4318.0,4522.0,3585.0,4196.0,4618.0,4154.0,3602.0,3685.0,4401.0,3206.0,Отклонение с высокой значимостью,4120.111111,4196.0,-22.186565,-23.593899,-2.028596,2023-10-20 09:28:46
189,B,210165,"ООО ""ЭЛАСТОМЕРИК СИСТЕМС""",1354.0,1471.0,1997.0,2470.0,2483.0,2209.0,2491.0,2103.0,2263.0,954.0,Отклонение с высокой значимостью,2093.444444,2209.0,-54.429170,-56.813038,-2.232142,2023-10-20 09:28:46
178,B,208842,Вимкор,1114.0,1128.0,1257.0,874.0,1250.0,1640.0,1411.0,1094.0,1197.0,736.0,Отклонение с высокой значимостью,1218.333333,1197.0,-39.589603,-38.512949,-2.002216,2023-10-20 09:28:46
33,B,128752,Компания МОССНАБСТРОЙ,744.0,819.0,743.0,720.0,926.0,818.0,540.0,743.0,516.0,456.0,Отклонение с высокой значимостью,729.888889,743.0,-37.524737,-38.627187,-1.917021,2023-10-20 09:28:46
255,C,214885,Родник Строй,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Отклонение с высокой значимостью,1.000000,1.0,-100.000000,-100.000000,-3.333333,2023-10-20 09:28:46
230,C,212889,Нержавеющая сталь,564.0,695.0,638.0,601.0,723.0,500.0,528.0,558.0,442.0,328.0,Отклонение с высокой значимостью,583.222222,564.0,-43.760716,-41.843972,-2.283708,2023-10-20 09:28:46
247,C,214472,СТИЛОТ,562.0,672.0,841.0,563.0,648.0,788.0,680.0,730.0,672.0,449.0,Отклонение с высокой значимостью,684.000000,672.0,-34.356725,-33.184524,-2.159615,2023-10-20 09:28:46


In [43]:
ws = gc.open('Вчерашнее_Критическое_падение_показателей').worksheet('base_vis')
ws.clear()
gd.set_with_dataframe(worksheet=ws,dataframe=high_descrease,include_index=False,include_column_header=True,resize=True)
# записываем полученную таблицу в файл гугл-таблиц

In [44]:
async def send_message(chat_id, text):
    bot = telegram.Bot(token='6460488253:AAGkAjh07JitjKMx05WJafhu90atH1dVFbA')
    await bot.send_message(chat_id=chat_id, text=text)

async def main():
    crit_lst = high_descrease.iloc[:5,2].tolist()
    message = f"Значительное падение просмотров по компаниям: {crit_lst}. Все компании с критичным падением - в файле https://docs.google.com/spreadsheets/d/1FtZDDK0_vJUc7gegkG9CRX8-MyxKZN_nH2sX_xP6hKQ/edit?usp=sharing"
    chat_id = -1001938632565

    await send_message(chat_id, message)
# передаем сообщение о критическом падении в телеграм-чат аккаунтов

In [45]:
asyncio.run(main())